In [ ]:
%cd /content/drive/MyDrive
!tar -xvzf bacteria.tar.gz

/content/drive/MyDrive
dataset/
dataset/Acinetobacter_baumanii/
dataset/Actinomyces_israeli/
dataset/Bacteroides_fragilis/
dataset/Bifidobacterium_spp/
dataset/Candida_albicans/
dataset/Clostridium_perfringens/
dataset/Enterococcus_faecium/
dataset/Lactobacillus_casei/
dataset/Neisseria_gonorrhoeae/
dataset/Porfyromonas_gingivalis/
dataset/Porfyromonas_gingivalis/Porfyromonas.gingivalis_0001_patch_0.tif
dataset/Porfyromonas_gingivalis/Porfyromonas.gingivalis_0001_patch_1.tif
dataset/Porfyromonas_gingivalis/Porfyromonas.gingivalis_0001_patch_2.tif
dataset/Porfyromonas_gingivalis/Porfyromonas.gingivalis_0001_patch_3.tif
dataset/Porfyromonas_gingivalis/Porfyromonas.gingivalis_0001_patch_4.tif
dataset/Porfyromonas_gingivalis/Porfyromonas.gingivalis_0001_patch_5.tif
dataset/Porfyromonas_gingivalis/Porfyromonas.gingivalis_0002_patch_0.tif
dataset/Porfyromonas_gingivalis/Porfyromonas.gingivalis_0002_patch_1.tif
dataset/Porfyromonas_gingivalis/Porfyromonas.gingivalis_0002_patch_2.tif
dataset/P

In [17]:
import os
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [18]:


# Define your dataset directory
dataset_dir = r'/content/drive/MyDrive/dataset'

# Define image size and batch size
img_height, img_width = 640, 640
batch_size = 32

# Create an ImageDataGenerator for data augmentation and preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # Splitting data into training and validation sets
)

# Load and prepare training data
train_data = train_datagen.flow_from_directory(
    dataset_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

# Load and prepare validation data
val_data = train_datagen.flow_from_directory(
    dataset_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)



Found 1040 images belonging to 10 classes.
Found 255 images belonging to 10 classes.


In [19]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import VGG16

# Define your dataset directory and other parameters
# dataset_dir = 'dataset'
img_height, img_width = 640, 640
num_classes = 10  # Specify the number of classes in your dataset

# Load the pre-trained VGG16 model without the top classification layer
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

# Freeze the weights of the VGG16 base model
base_model.trainable = False

# Create the modified model
inputs = layers.Input(shape=(img_height, img_width, 3))
x = base_model(inputs, training=False)
x = layers.Flatten()(x)
x = layers.Dropout(0.5)(x)  # Add dropout for regularization
outputs = layers.Dense(num_classes, activation='softmax')(x)

# Create the model
model = models.Model(inputs=inputs, outputs=outputs)

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Display the model summary
model.summary()


Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_16 (InputLayer)       [(None, 640, 640, 3)]     0         
                                                                 
 vgg16 (Functional)          (None, 20, 20, 512)       14714688  
                                                                 
 flatten_4 (Flatten)         (None, 204800)            0         
                                                                 
 dropout_6 (Dropout)         (None, 204800)            0         
                                                                 
 dense_6 (Dense)             (None, 10)                2048010   
                                                                 
Total params: 16762698 (63.94 MB)
Trainable params: 2048010 (7.81 MB)
Non-trainable params: 14714688 (56.13 MB)
_________________________________________________________________


In [4]:
# Train the model
history = model.fit(
    train_data,
    epochs=5,
    batch_size=batch_size,
    validation_data=val_data
)


Epoch 1/5
33/33 [==============================] - 1218s 36s/step - loss: 10.6044 - accuracy: 0.4010 - val_loss: 4.3979 - val_accuracy: 0.5961
Epoch 2/5
33/33 [==============================] - 242s 7s/step - loss: 0.8999 - accuracy: 0.8337 - val_loss: 0.7303 - val_accuracy: 0.8118
Epoch 3/5
33/33 [==============================] - 238s 7s/step - loss: 0.2994 - accuracy: 0.9260 - val_loss: 0.5666 - val_accuracy: 0.8353
Epoch 4/5
33/33 [==============================] - 239s 7s/step - loss: 0.2236 - accuracy: 0.9356 - val_loss: 0.6328 - val_accuracy: 0.8314
Epoch 5/5
33/33 [==============================] - 240s 7s/step - loss: 0.2028 - accuracy: 0.9413 - val_loss: 0.9917 - val_accuracy: 0.8471
